In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import glob
import datetime
import lxml.etree as ET
from xml.dom import minidom
import datetime
%matplotlib inline

In [ ]:
start = datetime.datetime.now()


In [ ]:
def time_elapsed(start):
    end = datetime.datetime.now()

    time_to_run = end - start
    minutes = int(time_to_run.seconds/60)
    seconds = time_to_run.seconds % 60
    print("Total runtime: ", minutes, "minutes, ", seconds, "seconds")

In [ ]:
fn = glob.glob("/data/arxiv/oai-af/*")

In [ ]:
len(fn)

In [ ]:
doc = minidom.parse(fn[2])
print(doc.toprettyxml())

In [ ]:
def get_author_text(doc):
    authorlist = []
    authortext = ""
    for author in doc.getElementsByTagName("authors")[0].childNodes:
        keyname = author.getElementsByTagName("keyname")[0].firstChild.data
        try:
            forenames = author.getElementsByTagName("forenames")[0].firstChild.data
        except:
            forenames = ""
        authortext += forenames + " " + keyname + ", "
        authorlist.append(forenames + " " + keyname)
    return authortext[:-2].replace("  ", " "), authorlist

In [ ]:
get_author_text(doc)

In [ ]:
df_list = []
dates = []
for file in fn[1:]:
    
    doc = minidom.parse(file)
    title = doc.getElementsByTagName("title")[0].firstChild.data
    arxivid = doc.getElementsByTagName("id")[0].firstChild.data
    created = doc.getElementsByTagName("created")[0].firstChild.data
    categories = doc.getElementsByTagName("categories")[0].firstChild.data
    abstract = doc.getElementsByTagName("abstract")[0].firstChild.data
    
    author_text, author_list = get_author_text(doc)
    
    num_authors = len(author_list)
    
    num_categories = len(categories.split(" "))
    
    try:
        updated = doc.getElementsByTagName("updated")[0].firstChild.data
    except IndexError:
        updated = np.nan
        
    try:
        doi = doc.getElementsByTagName("doi")[0].firstChild.data
    except IndexError:
        doi = np.nan 
        
        
    if 'cs.LG' in categories:
        cslg = 1
    else:
        cslg = 0
    
    if 'cs.AI' in categories:
        csai = 1
    else:
        csai = 0
    
    if 'stat.ML' in categories:
        statml = 1
    else:
        statml = 0
    
    if 'cs.CL' in categories:
        cscl = 1
    else:
        cscl = 0
        
    if 'cs.SI' in categories:
        cssi = 1
    else:
        cssi = 0
    
    if sum([cslg,csai,cscl,cssi,statml]) > 0:
        anymlcat = 1
    else:
        anymlcat = 0
        
        
    row = {
           'title':title,
           'updated':updated,
           'arxiv_id':arxivid,
           'created':created,
           'categories':categories.split(" "),
           'num_categories':num_categories,
           'abstract':abstract,
           'doi':doi,
           'cs.LG':cslg,
           'cs.AI':csai,
           'cs.CL':cscl,
           'cs.SI':cssi,
           'stat.ML':statml,
           'anymlcat':anymlcat,
           'author_text':author_text,
           'num_authors':num_authors
    }
    
    df_list.append(row)
    
    if len(df_list) % 25000 is 0:
        print(len(df_list), time_elapsed(start))

In [ ]:
df = pd.DataFrame(df_list)

In [ ]:
len(df)

In [ ]:
print(df.query("arxiv_id == '1207.2757'").author_text)

In [ ]:
df_anyml = df.query("anymlcat == 1")

In [ ]:
len(df_anyml)

In [ ]:
df.to_csv("processed_data/arxiv-oai-af.tsv", sep="\t")
df.to_hdf("processed_data/arxiv-oai-af.h5", key='df')


In [ ]:
df_anyml.to_csv("processed_data/arxiv-oai-af-anyml.tsv", sep="\t")
df_anyml.to_hdf("processed_data/arxiv-oai-af-anyml.h5", key='df')

In [ ]:
end = datetime.datetime.now()

time_to_run = end - start
minutes = int(time_to_run.seconds/60)
seconds = time_to_run.seconds % 60
print("Total runtime: ", minutes, "minutes, ", seconds, "seconds")
